In [6]:
################################################################
#### CALCULATING CLUSTERING STATS FOR POLLUTION_YEAR/grams #####
################################################################

# To do the analysis for *Capital_Amenity* change variable at 3 places where shown in code 
# and column names as to not overwrite existing data

# Steps: 
# 1. Read in attributes as CSV 
# 2. Read in Camden boundaries as GEOJSON 
# 3. Transform attributes to GEOJSON 
# 4. Calculate KNN weights (same results if you use boundaries or attributes)


# References:

# Spatial analysis: 
# http://darribas.org/gds_scipy16/ipynb_md/04_esda.html
# https://methods.sagepub.com/dataset/howtoguide/local-morans-i-berlin-districts-2018-python

# Function to transform csv to geojson:
# https://gis.stackexchange.com/questions/220997/pandas-to-geojson-multiples-points-features-with-python

# KNN justification 
# https://towardsdatascience.com/a-simple-introduction-to-k-nearest-neighbors-algorithm-b3519ed98e

# About Local Moran 
# https://pro.arcgis.com/en/pro-app/tool-reference/spatial-statistics/h-how-spatial-autocorrelation-moran-s-i-spatial-st.htm



import geojson
import pandas 
import geopandas
import pysal

# Reading in attributes data (Camden Tree data)
attributes_csv = pandas.read_csv('cleaned.csv')

# Aggregating by ward 
attributes_agg = attributes_csv.groupby(['Ward_Name'], as_index=False).mean()

# Function that will transform the csv pandas df to a geojson 
def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["Lat"],
                                                    X["Lon"])),
                            properties=dict(Ward=X["Ward_Name"],
                                            Amenity_Value=X["Amenity_Value"],
                                            Pollution_Year_grams=X["Pollution_Year_grams"])))
    df.apply(insert_features, axis=1)
    with open('attributes.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

# Applying function 
data2geojson(attributes_agg)

# Reading the attributes data as a geojson 
attributes_geojson = geopandas.read_file('attributes.geojson').to_crs(epsg = 27700)
attributes_geojson

# Reading in ward boundaries as a geojson 
neighbourhoods = geopandas.read_file('Camden Ward Boundary.geojson').to_crs(epsg = 27700) # geometry is a point 

# Calculating weights 'w' using the Camden Ward Boundary (NB: used with which has the centroid coords of each ward)
w_pollution = pysal.weights.KNN.from_dataframe(neighbourhoods, k=5)

# Calculating local moran statistics of proximity of *Pollution_Year_grams* using the weights 
local_morans = pysal.esda.moran.Moran_Local(attributes_geojson.Pollution_Year_grams, w_pollution, permutations=9999) 

# Checking what is significant 
Lag_pol = pysal.lag_spatial(w_pollution, attributes_geojson.Pollution_Year_grams) #*Change variable here*
polperyr = attributes_geojson.Pollution_Year_grams.values #*Change variable here*

sigs = polperyr[local_morans.p_sim <= .001]
W_sigs = Lag_pol[local_morans.p_sim <= .001]
insigs = polperyr[local_morans.p_sim > .001]
W_insigs = Lag_pol[local_morans.p_sim > .001]

# Calculating hotspots and coldspots 
sig = local_morans.p_sim < 0.05
hotspots = local_morans.q==1 * sig
hotspots.sum()
coldspots = local_morans.q==3 * sig
coldspots.sum()

# Assigning local moran values, coldspots and hotspots to: 
# 1. the attributes df 
# 2. the Camden geojson boundary 

# 1.
attributes_agg2 = attributes_agg.assign(pol_cl_value =local_morans.Is)
attributes_agg2 = attributes_agg2.assign(pol_hotspots = local_morans.q==1 * sig)
attributes_agg2 = attributes_agg2.assign(pol_coldspots = local_morans.q==3 * sig)

# 2. 
attributes_geojson2 = attributes_geojson.assign(pol_cl_value =local_morans.Is)
attributes_geojson2 = attributes_geojson2.assign(pol_hotspots = local_morans.q==1 * sig)
attributes_geojson2 = attributes_geojson2.assign(pol_coldspots = local_morans.q==3 * sig)

# Another statistic that can be used as the variable to be measured: z_sim
# z_sim = standardized Is based on permutations
# Assigning it to the two objects:
# Breaks for mapping this variable: 2.58 - 1.96 - 1.65 - -1.65 - -1.96 - -2.58; 

attributes_agg2 = attributes_agg2.assign(pol_z_sim = local_morans.z_sim)
attributes_geojson2 = attributes_geojson2.assign(pol_z_sim =local_morans.z_sim)

# THE END: there are two objects with the same data: 
# attributes_agg2 - attributes data and clustering stats in pandas df format.
# attributes_geojson2 - attributes data and clustering stats in GeoJSON.  

# WHAT SHOULD BE MAPPED: 
# Either: coldspots and hotspots (but there are no coldspots, so the map might be ugly, only red and greyish)
# Or pol_z_sim with breaks mentioned above, like this map can have a gradient (like the one made in R) 
# though I am not sure what the breaks mean, (they are from the GIS prac), I think they refer to the 
# 'intensity' of the clustering. 